In [1]:
import pandas as pd
from bs4 import BeautifulSoup 
from selenium import webdriver
import lxml
import lxml.html.soupparser
import re
import requests
import json
import random

In [2]:
#what i have to do:
  #  1. make functions out of the pieces of code that i already have here
  #  2. write comments for the things i do here
  #  3. understand how to output in the JSON format
  # 4. make it output info in the JSON format
  #  5. load the code on repl.it
  #  6. test on different items
  #  7. improve speed

In [3]:
# identifying the variables
products = []
headers = {
        'accept-encoding': 'gzip, deflate, sdch, br',
        'accept-language': 'en-US,en;q=0.8,zh-CN;q=0.6,zh;q=0.4',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'cache-control': 'max-age=0',
    }
base_url = "https://www.amazon.com/s?k="
#proxies_list = ["128.199.109.241:8080","113.53.230.195:3128","125.141.200.53:80","125.141.200.14:80","128.199.200.112:138","149.56.123.99:3128","128.199.200.112:80","125.141.200.39:80","134.213.29.202:4444"]
#proxies = {'https': random.choice(proxies_list)}

In [4]:
# function to gather the links to the products' pages
def linkc(item):
    # put together the url to scrape from
    url = base_url + item
    print('begin scraping the data', url)
    # get the page data
    request = requests.get(url, headers=headers)
    soup = BeautifulSoup(request.text, 'lxml')
    #print(soup)
    body = soup.find('body')
    main_outlet = body.find('div',{'class':"s-main-slot s-result-list s-search-results sg-row"})
    #print(main_outlet)
    links = main_outlet.find_all('a',{'class':'a-link-normal a-text-normal'})
    # get the links to pages
    for i in links:
        products.append('https://www.amazon.com' + i.get('href'))
#print(products[2])
#print(products)

In [19]:
# lists to store the data
prices = []
review_nums = []
review_scores = []
stock = []
titles = []
imgs = []
k = 1
def pagec(products,prices,review_nums,review_scores,stock,titles,imgs,k):
    # begin scraping
    for page in products:
        print('begin scrapping the product page: ', page)
        print('page number',k,'--- left:',len(products)-k)
        k = k + 1

        # get the page
        request = requests.get(page, headers=headers)
        soup1 = BeautifulSoup(request.text, 'lxml')
        #print(soup1)
        #body1 = soup1.find('body')

        # get the title of the item
        titles.append(soup1.find(id='productTitle').get_text().strip())
        print('the title:',titles[-1])

        # get the price
        try:
            prices.append(float(soup1.find('span',{'id':re.compile('^priceblock_')}).get_text().replace('$', '').replace(',', '.').strip()))
        except:
            try:
                prices.append(float(soup1.find('span',{'class':re.compile('^price')}).get_text().replace('$', '').replace(',', '.').strip()))
            except:
                prices.append('')
        print('the price:',prices[-1],'$')

        # review score
        #review_scores.append(float(soup1.select('.a-star-4-5')[0].get_text().split(' ')[0].replace(",", ".")))
        review_scores.append(float(soup1.find('span',{'class':'a-icon-alt'}).get_text().strip()[:3]))
        print('review score:',review_scores[-1])

        # how many reviews
        review_nums.append(int(soup1.select('#acrCustomerReviewText')[0].get_text().split(' ')[0].replace(".", "").replace(",","")))
        print('the number of reviews:', review_nums[-1])

        # whether it is available
        try:
            soup1.select('#availability .a-color-state')[0].get_text().strip()
            stock.append('Out of Stock')
        except:
            stock.append('Available')
        print(stock[-1])

         # get the link to the source image
        try:
            imgs.append(soup1.find('img',{'alt':titles[-1]}).get('data-old-hires').strip())
        except:
            imgs.append('')
        print('link to the image:',imgs[-1])

In [20]:
# get the input
item = input()
# lowercase the input and format it
item = item.lower().strip().replace(' ','+')
# run the functions
linkc(item)
pagec(products,prices,review_nums,review_scores,stock,titles,imgs,k)

men sunglasses
begin scraping the data https://www.amazon.com/s?k=men+sunglasses
begin scrapping the product page:  https://www.amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A003621531BDYFXBSBD74&url=%2FANYLUV-Sunglasses-Semi-Rimless-Glasses%25EF%25BC%2588No-Polarized%25EF%25BC%2589%2Fdp%2FB08B8TDCZ8%2Fref%3Dsr_1_1_sspa%3Fdchild%3D1%26keywords%3Dmen%2Bsunglasses%26qid%3D1598190461%26sr%3D8-1-spons%26psc%3D1&qualifier=1598190460&id=3452506129125195&widgetName=sp_atf
page number 1 --- left: 512
the title: Polarized Sunglasses Classic Men Women Retro Brand Designer Sun Glasses（No Slip, No Bounce, All Polarized）
the price: 15.99 $
review score: 4.7
the number of reviews: 16
Available
link to the image: https://images-na.ssl-images-amazon.com/images/I/61GMidiixmL._AC_UL1500_.jpg
begin scrapping the product page:  https://www.amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A03609742814H7MC76WIU&url=%2FATTCL-Driving

['RIVBOS Polarized Sports Sunglasses for Women Men Driving shades Cycling Running Rb833', 'SOJOS Classic Round Retro Plastic Frame Vintage Large Sunglasses BLOSSOM SJ2067', 'SOJOS Polarized Sunglasses Classic Small Round Metal Frame for Women Men SJ1014', 'SUNGAIT Vintage Round Sunglasses for Women Classic Retro Designer Style', 'SOJOS Small Square Polarized Sunglasses for Men and Women Polygon Mirrored Lens SJ1072', 'SUNGAIT Ultra Lightweight Rectangular Polarized Sunglasses UV400 Protection', 'SOJOS Fashion Round Sunglasses for Women Men Oversized Vintage Shades SJ2057', 'WearMe Pro - Reflective Lens Round Trendy Sunglasses', "Polarspex Polarized 80's Retro Classic Trendy Stylish Sunglasses for Men Women", 'Sunglasses for Women Men Polarized uv Protection Wearpro Fashion Vintage Round Classic Retro Aviator Mirrored Sun glasses', "MERRY'S Unisex Polarized Aluminum Sunglasses Vintage Sun Glasses For Men/Women S8286", 'SOJOS Classic Aviator Mirrored Flat Lens Sunglasses Metal Frame with

In [21]:
jj = json.dumps({'title':titles,'price':prices,'availability':stock,'score':review_scores,'number of reviews':review_nums,'image':imgs},indent=4)
print(jj)

{
    "title": [
        "Polarized Sunglasses Classic Men Women Retro Brand Designer Sun Glasses\uff08No Slip, No Bounce, All Polarized\uff09",
        "ATTCL Men's Retro Metal Frame Driving Polarized Sunglasses For Men",
        "Sports Polarized Sunglasses For Men Cycling Driving Fishing 100% UV Protection",
        "DUCO Men's Luxury Carbon Fiber Temple Polarized Sunglasses for Men Sports UV400 DC8206",
        "SUNGAIT Ultra Lightweight Rectangular Polarized Sunglasses UV400 Protection",
        "Polarized Sunglasses for Men Retro - FEIDU Polarized Retro Sunglasses for Men FD2149",
        "MERRY'S Unisex Polarized Aluminum Sunglasses Vintage Sun Glasses For Men/Women S8286"
    ],
    "price": [
        15.99,
        25.0,
        19.97,
        35.0,
        15.99,
        11.98,
        12.91
    ],
    "availability": [
        "Available",
        "Available",
        "Available",
        "Available",
        "Available",
        "Available",
        "Available"
    ],
    "

In [22]:
print(stock[0],prices[0],titles[0])

Available 15.99 Polarized Sunglasses Classic Men Women Retro Brand Designer Sun Glasses（No Slip, No Bounce, All Polarized）


In [32]:
# creating the DataFrame
info = {'title':titles,'price':prices,'availability':stock,'score':review_scores,'number of reviews':review_nums,'image':imgs}
df = pd.DataFrame.from_dict(info,orient='columns')
df.to_json(r'D:\Amazon_DataFrame.json')